# DuckDB Python Client Basics

This guide is based on the DuckDB documentation here: https://duckdb.org/docs/api/python/overview. In some parts, it takes directly from the example code.

In [135]:
import duckdb
import pandas as pd
import numpy as np
import polars as pl
import pyarrow as pa
import os.path

print("duckdb version: {}".format(duckdb.__version__))

duckdb version: 0.10.0


# Basic Python API Usage

In [136]:
duckdb.sql("SELECT 1, 2.0, 'a', 'TRUE'").show()

┌───────┬──────────────┬─────────┬─────────┐
│   1   │     2.0      │   'a'   │ 'TRUE'  │
│ int32 │ decimal(2,1) │ varchar │ varchar │
├───────┼──────────────┼─────────┼─────────┤
│     1 │          2.0 │ a       │ TRUE    │
└───────┴──────────────┴─────────┴─────────┘



In [137]:
relation = duckdb.sql(
    "SELECT 42 AS i"
)

(duckdb
 .sql("SELECT i * 2 AS k FROM relation")
 .show()
)

┌───────┐
│   k   │
│ int32 │
├───────┤
│    84 │
└───────┘



# Data Input
We will focus on reading CSVs for now.

In [138]:
# reading in a CSV with the duckdb.read_csv() function
path = "data/green_infrastructure.csv"
duckdb.read_csv(path)

┌──────────────────────┬──────────┬───────────┬────────────┬───┬────────────┬────────────────────┬─────────────────┐
│       the_geom       │ Asset_ID │   GI_ID   │ DEP_Contra │ … │ Construc_1 │     Program_Ar     │   Status_Gro    │
│       varchar        │  int64   │  varchar  │  varchar   │   │  varchar   │      varchar       │     varchar     │
├──────────────────────┼──────────┼───────────┼────────────┼───┼────────────┼────────────────────┼─────────────────┤
│ POINT (-73.8116762…  │    94002 │ 1A        │ GQJA03-02  │ … │ Package-1  │ Right of Way (ROW) │ Constructed     │
│ POINT (-73.8122857…  │    94012 │ GS6A      │ GQJA03-02  │ … │ Package-1  │ Right of Way (ROW) │ Constructed     │
│ POINT (-73.8122344…  │    94017 │ GS8C      │ GQJA03-02  │ … │ Package-1  │ Right of Way (ROW) │ Constructed     │
│ POINT (-73.8120597…  │    94019 │ GS8E      │ GQJA03-02  │ … │ Package-1  │ Right of Way (ROW) │ Constructed     │
│ POINT (-73.8131019…  │    94021 │ 10A       │ GQJA03-02  │ … │

In [139]:
# reading in a CSV by opening a connection and copying data to a table
con = duckdb.connect()

con.sql(
    "CREATE TABLE green_infrastructure AS FROM read_csv('data/green_infrastructure.csv')"
)

con.sql(
    "SELECT * FROM green_infrastructure"
)

┌──────────────────────┬──────────┬───────────┬────────────┬───┬────────────┬────────────────────┬─────────────────┐
│       the_geom       │ Asset_ID │   GI_ID   │ DEP_Contra │ … │ Construc_1 │     Program_Ar     │   Status_Gro    │
│       varchar        │  int64   │  varchar  │  varchar   │   │  varchar   │      varchar       │     varchar     │
├──────────────────────┼──────────┼───────────┼────────────┼───┼────────────┼────────────────────┼─────────────────┤
│ POINT (-73.8116762…  │    94002 │ 1A        │ GQJA03-02  │ … │ Package-1  │ Right of Way (ROW) │ Constructed     │
│ POINT (-73.8122857…  │    94012 │ GS6A      │ GQJA03-02  │ … │ Package-1  │ Right of Way (ROW) │ Constructed     │
│ POINT (-73.8122344…  │    94017 │ GS8C      │ GQJA03-02  │ … │ Package-1  │ Right of Way (ROW) │ Constructed     │
│ POINT (-73.8120597…  │    94019 │ GS8E      │ GQJA03-02  │ … │ Package-1  │ Right of Way (ROW) │ Constructed     │
│ POINT (-73.8131019…  │    94021 │ 10A       │ GQJA03-02  │ … │

In [140]:
%cd data/

%ls

/Users/geribauer/duckdb-python-basics/data
data-dictionary.xlsx      data-download.ipynb       green_infrastructure.csv


In [141]:
# reading in a CSV via the duckdb.sql() method
duckdb.sql(
    """
    SELECT *
    FROM green_infrastructure.csv
    LIMIT 10
    """
)

┌──────────────────────┬──────────┬─────────┬────────────┬───┬────────────┬────────────────────┬─────────────┐
│       the_geom       │ Asset_ID │  GI_ID  │ DEP_Contra │ … │ Construc_1 │     Program_Ar     │ Status_Gro  │
│       varchar        │  int64   │ varchar │  varchar   │   │  varchar   │      varchar       │   varchar   │
├──────────────────────┼──────────┼─────────┼────────────┼───┼────────────┼────────────────────┼─────────────┤
│ POINT (-73.8116762…  │    94002 │ 1A      │ GQJA03-02  │ … │ Package-1  │ Right of Way (ROW) │ Constructed │
│ POINT (-73.8122857…  │    94012 │ GS6A    │ GQJA03-02  │ … │ Package-1  │ Right of Way (ROW) │ Constructed │
│ POINT (-73.8122344…  │    94017 │ GS8C    │ GQJA03-02  │ … │ Package-1  │ Right of Way (ROW) │ Constructed │
│ POINT (-73.8120597…  │    94019 │ GS8E    │ GQJA03-02  │ … │ Package-1  │ Right of Way (ROW) │ Constructed │
│ POINT (-73.8131019…  │    94021 │ 10A     │ GQJA03-02  │ … │ Package-1  │ Right of Way (ROW) │ Constructed │
│

In [142]:
# cd back to root directory and out of data/
%cd ../

/Users/geribauer/duckdb-python-basics


# DataFrames

In [143]:
ls

README.md                   duckdb-python-sql.ipynb
data/                       file.db
duckdb-python-basics.ipynb


In [144]:
# pandas
df = pd.DataFrame({"numbers": [1, 2, 3, 4]})

duckdb.sql("SELECT * FROM df")

┌─────────┐
│ numbers │
│  int64  │
├─────────┤
│       1 │
│       2 │
│       3 │
│       4 │
└─────────┘

In [145]:
# polars
df = pl.DataFrame({"numbers": [1, 2, 3, 4]})

duckdb.sql("SELECT * FROM df")

┌─────────┐
│ numbers │
│  int64  │
├─────────┤
│       1 │
│       2 │
│       3 │
│       4 │
└─────────┘

In [146]:
# pyarrow
df = pa.Table.from_pydict({"numbers": [1, 2, 3, 4]})

duckdb.sql("SELECT * FROM df")

┌─────────┐
│ numbers │
│  int64  │
├─────────┤
│       1 │
│       2 │
│       3 │
│       4 │
└─────────┘

# Result Conversion

In [147]:
# Python objects
print(type(duckdb.sql("SELECT 1 as num").fetchall()))

# NumPy Arrays
print(type(duckdb.sql("SELECT 1 as num").fetchnumpy())) 

# Pandas DataFrame
print(type(duckdb.sql("SELECT 1 as num").df()))

# Polars DataFrame
print(type(duckdb.sql("SELECT 1 as num").pl()))

# Arrow Table
print(type(duckdb.sql("SELECT 1 as num").arrow()))

<class 'list'>
<class 'dict'>
<class 'pandas.core.frame.DataFrame'>
<class 'polars.dataframe.frame.DataFrame'>
<class 'pyarrow.lib.Table'>


In [148]:
# Example: pandas DataFrame
duckdb.sql("SELECT 1 as num").df()

,num
0,1


# Writing Data to Disk

In [149]:
# Write to a Parquet file
duckdb.sql("SELECT 42").write_parquet("data/out.parquet") 

# Write to a CSV file
duckdb.sql("SELECT 42").write_csv("data/out.csv")   

# Copy to a Parquet file
duckdb.sql("COPY (SELECT 42) TO 'data/out.parquet'")      

In [150]:
ls data/

data-dictionary.xlsx      green_infrastructure.csv  out.parquet
data-download.ipynb       out.csv


# Connection Options

## Using an In-Memory Database

In [151]:
con = duckdb.connect()

(con
 .sql("SELECT 1 AS num, 'a' AS letter")
 .show()
)

┌───────┬─────────┐
│  num  │ letter  │
│ int32 │ varchar │
├───────┼─────────┤
│     1 │ a       │
└───────┴─────────┘



A connection to a persistent database can be created using the `connect` function.

## Persistent Storage

In [152]:
con = duckdb.connect("data/file.db")

con.sql("CREATE TABLE IF NOT EXISTS integers (i INTEGER)")
con.sql("INSERT INTO integers VALUES (1), (2)")

con.sql("SELECT * FROM integers").show()

┌───────┐
│   i   │
│ int32 │
├───────┤
│     1 │
│     2 │
└───────┘



In [153]:
con = duckdb.connect("data/file.db")

con.sql("SELECT * FROM integers").show()

┌───────┐
│   i   │
│ int32 │
├───────┤
│     1 │
│     2 │
└───────┘



In [154]:
# create a connection to a file called 'file.db'
con = duckdb.connect("data/file.db")

con.sql("INSERT INTO integers VALUES (3)")

# query the table
con.table("integers").show()

# explicitly close the connection
con.close()
# Note: connections also closed implicitly when they go out of scope

┌───────┐
│   i   │
│ int32 │
├───────┤
│     1 │
│     2 │
│     3 │
└───────┘



In [155]:
ls

README.md                   duckdb-python-sql.ipynb
data/                       file.db
duckdb-python-basics.ipynb


In [156]:
with duckdb.connect("data/file.db") as con:
    con.sql("INSERT INTO integers VALUES (4)")
    con.table("integers").show()
    # the context manager closes the connection automatically

┌───────┐
│   i   │
│ int32 │
├───────┤
│     1 │
│     2 │
│     3 │
│     4 │
└───────┘



# Loading and Installing Extensions
Overview of extensions: https://duckdb.org/docs/extensions/overview

Note: I had to build this extension [from source](https://duckdb.org/docs/dev/building/overview.html), but make sure to thoroughly review DuckDB's extension guides.

In [158]:
con = duckdb.connect(config={"allow_unsigned_extensions": "true"})
path = "../duckdb/build/release/repository/v0.10.0/osx_amd64/httpfs.duckdb_extension"

con.install_extension(path)
con.load_extension(path)

# nyc taxi data
con.sql(
    """
    SELECT *
    FROM 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-01.parquet'
    LIMIT 10
    """
)

┌──────────┬──────────────────────┬──────────────────────┬───┬──────────────┬──────────────────────┬─────────────┐
│ VendorID │ tpep_pickup_datetime │ tpep_dropoff_datet…  │ … │ total_amount │ congestion_surcharge │ Airport_fee │
│  int32   │      timestamp       │      timestamp       │   │    double    │        double        │   double    │
├──────────┼──────────────────────┼──────────────────────┼───┼──────────────┼──────────────────────┼─────────────┤
│        2 │ 2024-01-01 00:57:55  │ 2024-01-01 01:17:43  │ … │         22.7 │                  2.5 │         0.0 │
│        1 │ 2024-01-01 00:03:00  │ 2024-01-01 00:09:36  │ … │        18.75 │                  2.5 │         0.0 │
│        1 │ 2024-01-01 00:17:06  │ 2024-01-01 00:35:01  │ … │         31.3 │                  2.5 │         0.0 │
│        1 │ 2024-01-01 00:36:38  │ 2024-01-01 00:44:56  │ … │         17.0 │                  2.5 │         0.0 │
│        1 │ 2024-01-01 00:46:51  │ 2024-01-01 00:52:57  │ … │         16.1 │   